## Import

In [1]:
!pip install sentence-transformers datasets

In [2]:
from sentence_transformers import SentenceTransformer
from pathlib import Path
from datasets import load_from_disk
import pandas as pd
import numpy as np
import json
import time
from tqdm import tqdm

## Dataset prepare

In [4]:
df = pd.read_json("/merged_email_dataset.json")
# df = pd.read_json("hf://datasets/0tt00t/PI-EmailGuard/merged_email_dataset.json")

In [5]:
output_column = df["output"].tolist()
sentences = output_column

## Embedding

In [6]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
batch_size = 1000
n = len(sentences)
emb_list = []
encode_time = 0.0

In [8]:
with tqdm(total=n, desc="Encoding", unit="items") as pbar:
    for i in range(0, n, batch_size):
        batch = sentences[i : i + batch_size]
        t0 = time.time()
        batch_emb = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
        t1 = time.time()
        encode_time += (t1 - t0)
        emb_list.append(batch_emb)
        pbar.update(len(batch))

if len(emb_list) > 0:
    embeddings = np.vstack(emb_list)
else:
    embeddings = np.empty((0, model.get_sentence_embedding_dimension()))

print(f"Time taken to generate embeddings (encode only): {encode_time:.2f} seconds")

Encoding: 100%|██████████| 2000/2000 [00:02<00:00, 780.80items/s]

Time taken to generate embeddings (encode only): 2.56 seconds


In [11]:
df.head(10)

,output,is_injected,id
0,Processed example output for prompt: Subject o...,True,0
1,Processed example output for prompt: Subject o...,True,1
2,Processed example output for prompt: Subject o...,True,2
3,Processed example output for prompt: Subject o...,True,3
4,Processed example output for prompt: Subject o...,True,4
5,Processed example output for prompt: Subject o...,True,5
6,Processed example output for prompt: Subject o...,True,6
7,Processed example output for prompt: Subject o...,True,7
8,Processed example output for prompt: Subject o...,True,8
9,Processed example output for prompt: Subject o...,True,9


## Prepare data for modeling


In [12]:
y = df['is_injected']
X = embeddings

## Split data



In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train a random forest classifier


In [14]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [18]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)


print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.9975
Precision: 1.0000
Recall: 0.9950
F1-score: 0.9975
Confusion Matrix:
[[201   0]
 [  1 198]]


## Train an SVM classifier

In [19]:
from sklearn.svm import SVC

svm_model = SVC(random_state=42)
svm_model.fit(X_train, y_train)

SVC(random_state=42)

## Evaluate the svm model


In [20]:
y_pred_svm = svm_model.predict(X_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)

print(f"SVM Accuracy: {accuracy_svm:.4f}")
print(f"SVM Precision: {precision_svm:.4f}")
print(f"SVM Recall: {recall_svm:.4f}")
print(f"SVM F1-score: {f1_svm:.4f}")
print(f"SVM Confusion Matrix:\n{conf_matrix_svm}")

SVM Accuracy: 0.9950
SVM Precision: 1.0000
SVM Recall: 0.9899
SVM F1-score: 0.9949
SVM Confusion Matrix:
[[201   0]
 [  2 197]]
